# Shoplite RAG System - Complete Deployment

**Week 3 Assignment - Joseph Chamoun**

This notebook is fully self-contained and implements a complete RAG system for Shoplite customer service.

## Features:
- ✅ 15 embedded knowledge base documents
- ✅ FAISS vector search with sentence-transformers
- ✅ Llama 3.1 8B (or fallback to smaller model)
- ✅ Structured YAML prompts integrated
- ✅ Flask API with /chat, /ping, /health endpoints
- ✅ ngrok tunnel with runtime token input
- ✅ Smart retrieval with confidence scoring

**IMPORTANT**: Make sure to select GPU runtime (Runtime → Change runtime type → GPU)

In [ ]:
# Cell 1: Install all dependencies
print("📦 Installing dependencies... (this may take 2-3 minutes)")
!pip install -q --upgrade pip
!pip install -q transformers accelerate bitsandbytes sentence-transformers faiss-cpu flask pyngrok
print("✅ Installation complete!")

In [ ]:
# Cell 2: Imports
import os
import time
import json
import threading
from typing import List, Dict, Any
import numpy as np
import torch
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from flask import Flask, request, jsonify
from pyngrok import ngrok

print("✅ All imports successful")
print(f"🔥 CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Cell 3: Knowledge Base (15 Shoplite documents embedded)
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration and Account Management",
        "content": "To create a Shoplite account, users must visit the registration page and provide a valid email address, password, and basic profile information. Email verification is required within 24 hours. Users can choose between buyer accounts (free) or seller accounts (requires business verification and tax information). Account Management Features include: Update personal information, Change passwords, Set security questions, Manage notification preferences, Deactivate accounts (requires email confirmation; may affect active orders/subscriptions). Buyer Access includes: product browsing, purchasing, order tracking, reviews. Seller Access includes: seller dashboard, inventory management, order processing, analytics. Security Measures: two-factor authentication recommended, password recovery via email and phone verification."
    },
    {
        "id": "doc2",
        "title": "Shoplite Product Search and Filtering Features",
        "content": "Shoplite provides a powerful search engine with keyword queries, category selection, and brand filters. Filtering Options include: price range, rating, availability, seller location, shipping speed, promotions, and eco-friendly options. Features include autocomplete suggestions, spelling correction, save searches and alerts, faceted navigation for combining multiple filters, optimization for large catalogs with real-time indexing, and a mobile responsive interface."
    },
    {
        "id": "doc3",
        "title": "Shoplite Shopping Cart and Checkout Process",
        "content": "Users can add multiple items from different sellers, review quantities, and apply promo codes or gift cards. The cart is preserved across sessions for logged-in users. Checkout Steps: 1) Shipping selection (standard, expedited, same-day), 2) Payment selection (credit/debit cards, digital wallets, cash-on-delivery), 3) Order confirmation. Security and Processing features include PCI-DSS compliant payment gateways, real-time stock updates, order confirmation emails with tracking, seller notifications for new orders, and an integrated returns and refunds system."
    },
    {
        "id": "doc4",
        "title": "Shoplite Payment Methods and Security",
        "content": "Accepted Payment Methods include credit/debit cards, PayPal, Apple Pay, Google Pay, and local payment solutions. Security Measures include SSL encryption, PCI-DSS compliance, fraud detection systems, two-factor authentication, and sensitive information encrypted both in transit and at rest. Other Features include digital wallet integration, a structured dispute and chargeback process, and seller payments processed after order confirmation."
    },
    {
        "id": "doc5",
        "title": "Shoplite Order Tracking and Delivery",
        "content": "Shoplite provides real-time tracking with confirmation emails and unique tracking numbers. Order Stages include: confirmed, processing, shipped, in transit, and delivered. Users can request delivery modifications (seller approval required). International shipments display customs and import duties information. The system uses optimized logistics with estimated arrival times and delay notifications. Support assistance is available for lost or delayed packages."
    },
    {
        "id": "doc6",
        "title": "Shoplite Return and Refund Policies",
        "content": "The return period is typically 30 days from delivery. Process: select the order and item, specify the reason, and use the prepaid label if eligible. Refunds are processed in 5–7 business days to the original payment method. Digital and personalized items may have exceptions. The system provides automated order status updates. Sellers must comply with return policies to maintain their ratings. Dispute resolution services are available for complex cases."
    },
    {
        "id": "doc7",
        "title": "Shoplite Product Reviews and Ratings",
        "content": "Buyers can rate products on a five-star scale and leave detailed comments. Reviews are moderated for compliance with community guidelines. Sellers can respond to reviews to address concerns. Ratings influence search ranking and product visibility. Verified purchase badges ensure review authenticity. Aggregate ratings are provided on product pages. Review analytics are available for sellers to track customer feedback trends."
    },
    {
        "id": "doc8",
        "title": "Shoplite Seller Account Setup and Management",
        "content": "To create a seller account, provide business documents and complete tax verification, which takes 2-3 business days. The Seller Dashboard includes inventory management, order processing, and sales analytics. Product listing is available via individual entry or bulk upload (CSV/API). Profile customization includes branding, policies, shipping options, and return policies. Sellers receive notifications for new orders, low stock alerts, and customer inquiries. Features include pricing management, promotional tools, and shipping fee configuration. Performance metrics are tracked, and third-party integrations are supported."
    },
    {
        "id": "doc9",
        "title": "Shoplite Inventory Management for Sellers",
        "content": "Sellers can track stock levels, set reorder thresholds, and manage availability in real-time. The system provides low-stock alerts to prevent stockouts. Bulk imports are supported for efficient inventory updates. Product variants (size, color, bundles) are fully supported. Inventory reports help identify trends and prepare for seasonal demand. Sellers can manage multiple warehouses and shipping locations from a single dashboard."
    },
    {
        "id": "doc10",
        "title": "Shoplite Commission and Fee Structure",
        "content": "Shoplite charges a standard 5% commission on each sale. Commission fees vary by product category. Additional fees may apply for premium listings, promotional campaigns, and special services. Transparent fee notifications are displayed in the seller dashboard. Payments are made after commission deduction on a weekly or bi-weekly schedule. Detailed transaction reports are available for accounting purposes. Pricing guidance helps sellers remain competitive."
    },
    {
        "id": "doc11",
        "title": "Shoplite Customer Support Procedures",
        "content": "Support is available via live chat, email, phone, and an AI chatbot, with 24/7 availability. Tickets are categorized by type: orders, payments, returns, technical issues, and account management. Each ticket receives a unique tracking ID. Backend integration provides instant access to order and payment information. A dedicated seller support channel addresses business-specific concerns. The help center includes comprehensive guides, FAQs, and video tutorials. The goal is fast, transparent, and fair resolution for all users."
    },
    {
        "id": "doc12",
        "title": "Shoplite Mobile App Features",
        "content": "The Shoplite mobile app supports iOS and Android devices. Users can browse products, apply filters, add items to cart, and complete purchases. Push notifications alert users to promotions and order updates. Features include barcode scanning and QR code payments for convenience. Mobile wallets, fingerprint authentication, and Face ID login enhance security. Sellers can manage their stores and process orders on-the-go. Offline caching allows users to view previously loaded content without an internet connection. The interface is intuitive, responsive, and accessible."
    },
    {
        "id": "doc13",
        "title": "Shoplite API Documentation for Developers",
        "content": "Shoplite provides RESTful API endpoints for product catalog access, order management, account operations, and inventory updates. Authentication uses OAuth 2.0 for secure access. Rate limiting applies to prevent abuse, with higher limits for verified partners. Detailed documentation includes request/response examples, parameter descriptions, and error code explanations. Webhooks enable real-time event notifications for orders, inventory changes, and payments. A sandbox environment is available for testing without affecting live data. The API is versioned to ensure backward-compatible updates."
    },
    {
        "id": "doc14",
        "title": "Shoplite Security and Privacy Policies",
        "content": "Data Protection includes TLS encryption for data in transit and AES-256 encryption for data at rest. Access is restricted to authorized personnel only. Two-factor authentication and strong password requirements protect user accounts. Shoplite complies with GDPR and CCPA regulations. Security monitoring detects suspicious activity in real-time. Clear privacy policies outline data collection, usage, and third-party sharing practices. Users are notified of policy changes and maintain control over their personal data, including the ability to download or delete their information."
    },
    {
        "id": "doc15",
        "title": "Shoplite Promotional Codes and Discounts",
        "content": "Sellers can create promotions including discount codes, seasonal sales, and bundle offers. Code types include percentage discounts, fixed amount discounts, and conditional discounts. Configuration options include start/end dates, usage limits, and minimum purchase requirements. Automatic verification occurs at checkout to ensure eligibility. Analytics track redemption rates, revenue impact, and customer engagement. Users receive notifications about active promotions they qualify for. Special events are highlighted on the homepage and mobile app. All promotions must comply with platform policies to ensure fairness."
    }
]

print(f"✅ Knowledge base loaded: {len(KNOWLEDGE_BASE)} documents")

In [ ]:
# Cell 4: Structured Prompts (converted from YAML)
PROMPTS = {
    "version": "1.0",
    "created": "2025-09-26",
    "author": "Joseph Chamoun",
    
    "base_retrieval_prompt": {
        "role": "You are a knowledgeable Shoplite customer service assistant.",
        "goal": "Provide accurate, concise answers using only the provided Shoplite documentation.",
        "context_guidelines": [
            "Use only information from the provided document snippets",
            "Cite specific documents when possible",
            "Keep answers focused and relevant",
            "If information is unclear, acknowledge limitations"
        ],
        "response_format": "Provide a clear, direct answer in 2-3 sentences based on the context."
    },
    
    "multi_doc_synthesis": {
        "role": "You are an expert Shoplite support agent who synthesizes information from multiple documents.",
        "goal": "Combine relevant information from multiple sources to create a comprehensive, accurate answer.",
        "context_guidelines": [
            "Retrieve and integrate information from all relevant documents",
            "Provide step-by-step guidance if needed",
            "Avoid adding information not present in the documents",
            "Maintain consistency across different document sources"
        ],
        "response_format": "Synthesize information from multiple sources into a coherent answer."
    },
    
    "clarification_prompt": {
        "role": "You are a helpful Shoplite assistant that seeks clarity when needed.",
        "goal": "Ask for clarification politely when the user query is unclear or insufficient.",
        "context_guidelines": [
            "Do not guess answers if the query is unclear",
            "Suggest specific questions or information needed",
            "Remain helpful and professional"
        ],
        "response_format": "Politely ask for clarification with specific guidance."
    },
    
    "refusal_prompt": {
        "role": "You are a responsible Shoplite assistant that only answers when relevant context is available.",
        "goal": "Politely refuse to answer if the requested information is not found in the knowledge base.",
        "context_guidelines": [
            "Do not hallucinate information",
            "Provide guidance on where the user may find help",
            "Remain professional and helpful"
        ],
        "response_format": "Politely explain that the information is not available and suggest alternatives."
    }
}

print(f"✅ Loaded {len(PROMPTS) - 3} structured prompt configurations")

In [ ]:
# Cell 5: Build FAISS Index with Embeddings
print("🔄 Loading embedding model...")
EMBED_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

# Prepare documents for embedding
DOCUMENT_TEXTS = [f"{d['title']}\n\n{d['content']}" for d in KNOWLEDGE_BASE]
DOC_IDS = [d['id'] for d in KNOWLEDGE_BASE]

print("🔄 Creating embeddings...")
doc_embeddings = embed_model.encode(DOCUMENT_TEXTS, convert_to_numpy=True, show_progress_bar=True)

# Normalize for cosine similarity
def normalize_embeddings(embs):
    norms = np.linalg.norm(embs, axis=1, keepdims=True)
    return embs / np.clip(norms, a_min=1e-10, a_max=None)

doc_embeddings = normalize_embeddings(doc_embeddings)

# Build FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner product for cosine similarity
index.add(doc_embeddings)

print(f"✅ FAISS index built: {index.ntotal} vectors (dim={dimension})")

In [ ]:
# Cell 6: Retrieval Function
def retrieve_docs(query: str, top_k: int = 3):
    """Retrieve most relevant documents for a query."""
    q_emb = embed_model.encode([query], convert_to_numpy=True)
    q_emb = normalize_embeddings(q_emb)
    
    scores, indices = index.search(q_emb, top_k)
    
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if 0 <= idx < len(KNOWLEDGE_BASE):
            doc = KNOWLEDGE_BASE[idx]
            results.append({
                'id': doc['id'],
                'title': doc['title'],
                'content': doc['content'],
                'score': float(score)
            })
    
    return results

# Quick test
test_results = retrieve_docs("How do I create a seller account?", top_k=3)
print(f"✅ Retrieval test passed: Found {len(test_results)} relevant docs")
print(f"   Top match: {test_results[0]['title']} (score: {test_results[0]['score']:.3f})")

In [ ]:
# Cell 7: Load LLM with Quantization
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
FALLBACK_MODEL = "meta-llama/Llama-3.2-3B-Instruct"  # Smaller fallback

print(f"🔄 Loading model: {MODEL_NAME}")
print(f"💡 Fallback available: {FALLBACK_MODEL}")

use_cuda = torch.cuda.is_available()
print(f"{'🔥' if use_cuda else '⚠️'} CUDA available: {use_cuda}")

model = None
tokenizer = None
loaded_model_name = None

def try_load_model(model_name: str):
    """Attempt to load model with 8-bit quantization."""
    try:
        print(f"  Attempting: {model_name}")
        
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        
        # Configure 8-bit quantization
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_4bit_compute_dtype=torch.float16
        )
        
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto" if use_cuda else None,
            quantization_config=bnb_config if use_cuda else None,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        
        mdl.eval()
        print(f"  ✅ Success!")
        return tok, mdl, model_name
        
    except Exception as e:
        print(f"  ❌ Failed: {str(e)[:100]}")
        return None, None, None

# Try primary model
tokenizer, model, loaded_model_name = try_load_model(MODEL_NAME)

# Try fallback if primary fails
if model is None:
    print(f"\n🔄 Trying fallback model...")
    tokenizer, model, loaded_model_name = try_load_model(FALLBACK_MODEL)

# Final status
if model is not None:
    print(f"\n✅ MODEL LOADED: {loaded_model_name}")
    print(f"   Memory: ~{torch.cuda.memory_allocated() / 1e9:.1f} GB" if use_cuda else "   Running on CPU")
else:
    print("\n⚠️ No model loaded - will use extractive responses from retrieved docs")

In [ ]:
# Cell 8: Complete Generation Function with Structured Prompts
SIMILARITY_THRESHOLD = 0.35
MAX_NEW_TOKENS = 250
TEMPERATURE = 0.7

def build_prompt_from_retrieval(query: str, retrieved_docs: List[Dict[str, Any]], prompt_type: str = "base_retrieval_prompt"):
    """Build prompt using structured YAML prompts."""
    # Get prompt configuration
    prompt_config = PROMPTS.get(prompt_type, PROMPTS["base_retrieval_prompt"])
    
    # Format retrieved documents
    docs_text = ""
    for i, doc in enumerate(retrieved_docs[:3], 1):
        content = doc['content'][:300] + "..." if len(doc['content']) > 300 else doc['content']
        docs_text += f"\n[Document {i}: {doc['title']}]\n{content}\n"
    
    # Build structured prompt
    prompt = f"{prompt_config['role']}\n\n"
    prompt += f"Goal: {prompt_config['goal']}\n\n"
    prompt += "Guidelines:\n"
    for guideline in prompt_config['context_guidelines']:
        prompt += f"- {guideline}\n"
    prompt += f"\nContext:{docs_text}\n"
    prompt += f"Question: {query}\n\n"
    prompt += f"{prompt_config['response_format']}\n\nAnswer:"
    
    return prompt

def generate_response(query: str, top_k: int = 3, debug: bool = False):
    """Generate response using RAG pipeline."""
    
    # Retrieve relevant documents
    retrieved = retrieve_docs(query, top_k=top_k)
    
    if not retrieved:
        return {
            "answer": "I couldn't find relevant information in the Shoplite knowledge base. Please try rephrasing your question.",
            "sources": [],
            "confidence": "Low"
        }
    
    top_score = max(d['score'] for d in retrieved)
    
    # Determine confidence and prompt type
    if top_score < SIMILARITY_THRESHOLD:
        return {
            "answer": "I don't have specific information about that in my knowledge base. I can help with Shoplite's registration, orders, payments, returns, seller accounts, and customer support features.",
            "sources": [],
            "confidence": "Low"
        }
    
    # Set confidence level
    if top_score >= 0.65:
        confidence = "High"
        prompt_type = "base_retrieval_prompt"
    elif top_score >= 0.45:
        confidence = "Medium"
        prompt_type = "base_retrieval_prompt"
    else:
        confidence = "Low"
        prompt_type = "clarification_prompt"
    
    # Build prompt with structured format
    prompt = build_prompt_from_retrieval(query, retrieved, prompt_type)
    
    if debug:
        print(f"\n[DEBUG] Top score: {top_score:.3f}, Confidence: {confidence}")
        print(f"[DEBUG] Prompt length: {len(prompt)} chars")
    
    # If no model, use extractive approach
    if model is None:
        sentences = []
        for doc in retrieved[:2]:
            doc_sentences = [s.strip() + '.' for s in doc['content'].split('.') if len(s.strip()) > 30][:2]
            sentences.extend(doc_sentences)
        
        answer = ' '.join(sentences[:3])
        return {
            "answer": answer,
            "sources": [d['title'] for d in retrieved[:2]],
            "confidence": confidence
        }
    
    # Generate with LLM
    try:
        inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=1800)
        
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=TEMPERATURE,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.2,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        # Extract only the generated text
        generated_ids = outputs[0][inputs['input_ids'].shape[1]:]
        raw_text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        
        if debug:
            print(f"[DEBUG] Raw output: {raw_text[:150]}...")
        
        # Clean up the response - extract first 2-3 sentences
        sentences = []
        current = ""
        
        for char in raw_text:
            current += char
            if char in '.!?' and len(current.strip()) > 20:
                sentences.append(current.strip())
                current = ""
                if len(sentences) >= 3:
                    break
        
        answer = ' '.join(sentences[:3]) if sentences else raw_text[:300]
        
        # Remove incomplete final sentence
        if answer and not any(answer.endswith(p) for p in ['.', '!', '?']):
            last_period = max(answer.rfind('.'), answer.rfind('!'), answer.rfind('?'))
            if last_period > 0:
                answer = answer[:last_period + 1]
        
        # Fallback to extractive if generation too short
        if len(answer) < 30:
            doc = retrieved[0]
            sentences = [s.strip() + '.' for s in doc['content'].split('.') if len(s.strip()) > 25][:2]
            answer = ' '.join(sentences)
        
        return {
            "answer": answer,
            "sources": [d['title'] for d in retrieved[:3]],
            "confidence": confidence
        }
        
    except Exception as e:
        if debug:
            print(f"[DEBUG] Generation error: {e}")
        
        # Fallback to extractive
        doc = retrieved[0]
        sentences = [s.strip() + '.' for s in doc['content'].split('.') if len(s.strip()) > 25][:2]
        return {
            "answer": ' '.join(sentences),
            "sources": [d['title'] for d in retrieved[:2]],
            "confidence": "Medium"
        }

print("✅ Generation pipeline configured")

In [ ]:
# Cell 9: Flask API Setup
app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint."""
    return jsonify({
        "status": "running",
        "model_loaded": model is not None,
        "model_name": loaded_model_name if model else "None",
        "num_docs": len(KNOWLEDGE_BASE),
        "embedding_model": EMBED_MODEL_NAME
    })

@app.route('/ping', methods=['POST'])
def ping():
    """Direct LLM interaction without RAG."""
    try:
        data = request.get_json()
        query = data.get('query', '')
        
        if not query:
            return jsonify({"error": "No query provided"}), 400
        
        if model is None:
            return jsonify({"error": "Model not loaded"}), 503
        
        inputs = tokenizer(query, return_tensors='pt', truncation=True, max_length=512)
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        
        return jsonify({
            "response": response.strip(),
            "query": query
        })
        
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/chat', methods=['POST'])
def chat():
    """RAG-powered chat endpoint."""
    try:
        data = request.get_json()
        query = data.get('query', '')
        top_k = data.get('top_k', 3)
        
        if not query:
            return jsonify({"error": "No query provided"}), 400
        
        result = generate_response(query, top_k=top_k)
        
        return jsonify(result)
        
    except Exception as e:
        return jsonify({"error": str(e)}), 500

print("✅ Flask API configured with /health, /ping, /chat endpoints")

In [ ]:
# Cell 10: ngrok Tunnel Setup
print("🔐 ngrok Setup")
print("=" * 50)

# SECURITY: Accept token at runtime
ngrok_token = input("Enter your ngrok auth token: ").strip()

if not ngrok_token:
    print("❌ No token provided. Cannot create tunnel.")
else:
    try:
        # Set ngrok token
        ngrok.set_auth_token(ngrok_token)
        
        # Start Flask in background thread
        def run_flask():
            app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
        
        flask_thread = threading.Thread(target=run_flask, daemon=True)
        flask_thread.start()
        
        # Wait for Flask to start
        time.sleep(3)
        
        # Create ngrok tunnel
        public_url = ngrok.connect(5000, bind_tls=True)
        
        print("\n✅ TUNNEL ACTIVE")
        print("=" * 50)
        print(f"🌐 Public URL: {public_url}")
        print("=" * 50)
        print("\nAPI Endpoints:")
        print(f"  • Health: {public_url}/health")
        print(f"  • Chat:   {public_url}/chat")
        print(f"  • Ping:   {public_url}/ping")
        print("\n💡 Copy the URL above to use in chat-interface.py")
        print("⚠️  Keep this cell running - do not stop execution!")
        
    except Exception as e:
        print(f"❌ Tunnel creation failed: {e}")
        print("Please check your ngrok token and try again.")

In [ ]:
# Cell 11: System Validation and Testing
print("🧪 Running System Tests...\n")

# Test 1: Retrieval Quality
print("Test 1: Document Retrieval")
test_q = "How do I create a seller account?"
docs = retrieve_docs(test_q, top_k=3)
print(f"✅ Retrieved {len(docs)} docs")
print(f"   Top: {docs[0]['title']} (score: {docs[0]['score']:.3f})")
print(f"   Expected: 'Seller Account Setup and Management'")
assert "Seller" in docs[0]['title'], "Top document should be about seller accounts"

# Test 2: Response Generation
print("\nTest 2: Response Generation")
response = generate_response(test_q)
print(f"✅ Generated response ({len(response['answer'])} chars)")
print(f"   Answer preview: {response['answer'][:100]}...")
print(f"   Sources: {', '.join(response['sources'][:2])}")
print(f"   Confidence: {response['confidence']}")
assert len(response['answer']) > 20, "Response should be substantial"
assert response['confidence'] in ['High', 'Medium', 'Low'], "Invalid confidence level"

# Test 3: Multi-document Query
print("\nTest 3: Multi-Document Query")
multi_q = "What are the return policies and how do I track my order?"
response = generate_response(multi_q)
print(f"✅ Synthesized from {len(response['sources'])} sources")
print(f"   Sources: {', '.join(response['sources'])}")
assert len(response['sources']) >= 2, "Should retrieve multiple relevant documents"

# Test 4: Edge Case - Out of Scope
print("\nTest 4: Out-of-Scope Query")
oos_q = "What's the weather like today?"
response = generate_response(oos_q)
print(f"✅ Handled correctly: '{response['answer'][:80]}...'")
print(f"   Confidence: {response['confidence']}")
assert response['confidence'] == 'Low', "Out-of-scope should have low confidence"

# Test 5: Low Similarity Query
print("\nTest 5: Ambiguous Query")
amb_q = "Tell me about it"
response = generate_response(amb_q)
print(f"✅ Response: '{response['answer'][:80]}...'")
print(f"   Confidence: {response['confidence']}")

print("\n" + "="*50)
print("✅ ALL TESTS PASSED - System Ready for Deployment!")
print("="*50)
print("\n📋 System Summary:")
print(f"  • Knowledge Base: {len(KNOWLEDGE_BASE)} documents")
print(f"  • Embedding Model: {EMBED_MODEL_NAME}")
print(f"  • LLM: {loaded_model_name if model else 'Extractive Mode'}")
print(f"  • FAISS Index: {index.ntotal} vectors")
print(f"  • Prompt Templates: {len(PROMPTS) - 3} configured")
print("\n🚀 Ready to accept chat requests!")